# Filter Method Analysis

### Libraries

In [2]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from mafese import Data
from sklearn.model_selection import train_test_split
from mafese import FilterSelector
from mafese import get_dataset
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns

### data

In [3]:
og_data = pd.read_csv(r"C:\Users\Coord-Personal\Documents\ITZELT\data_sciece_girlypop\proyecto_salvador\datos\data.csv")

## exploratory analysis

In [4]:
og_data.columns

Index(['userid', 'label', 'meanX', 'meanY', 'meanZ', 'sdX', 'sdY', 'sdZ',
       'maxX', 'maxY', 'maxZ', 'minX', 'minY', 'minZ', 'corXY', 'corXZ',
       'corYZ', 'meanMagnitude', 'sdMagnitude', 'auc', 'meanDif', 'meanX2',
       'meanY2', 'meanZ2', 'sdX2', 'sdY2', 'sdZ2', 'maxX2', 'maxY2', 'maxZ2',
       'minX2', 'minY2', 'minZ2', 'corXY2', 'corXZ2', 'corYZ2',
       'meanMagnitude2', 'sdMagnitude2', 'auc2', 'meanDif2'],
      dtype='object')

This part was calculated in R. Basically, there are two sensors (accelerometers), on the back (1) and on the thigh (2). Each one has three types of orientations (front is Z, right side is Y, and bottom is X). Then, each numerical column represents the values ​​of these orientations, which are:

* means
* SD
* maximum and minimum value
* correlations between orientations
* the mean value of the magnitude
* the std. deviation of the magnitude

In [5]:
og_data["label"].unique()

array(['walking', 'shuffling', 'standing', 'sitting', 'lying'],
      dtype=object)

In [6]:
og_data.shape

(10683, 40)

In [7]:
og_data.groupby("userid")["label"].value_counts()

userid  label    
501     walking      419
        standing      95
        sitting       24
        shuffling     12
        lying          7
                    ... 
517     walking      496
        standing     163
        shuffling     29
        sitting       11
        lying          2
Name: count, Length: 85, dtype: int64

from what can be seen, the data for each class is really unbalanced

In [8]:
og_data["userid"].value_counts()

userid
504    822
516    811
512    752
517    701
508    680
507    657
513    651
515    645
510    598
511    597
502    584
506    578
509    559
501    557
503    554
514    515
505    422
Name: count, dtype: int64

**codified the variable to predict**

* walking: 1
* shuffling: 2
* standing: 3
* sitting: 4
* lying: 5

In [9]:
og_data["label"] = og_data["label"].replace({"walking": 1, "shuffling": 2, "standing": 3, "sitting": 4, "lying": 5})
og_data

C:\Users\Coord-Personal\AppData\Local\Temp\ipykernel_5788\789855610.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,userid,label,meanX,meanY,meanZ,sdX,sdY,sdZ,maxX,maxY,...,minX2,minY2,minZ2,corXY2,corXZ2,corYZ2,meanMagnitude2,sdMagnitude2,auc2,meanDif2
0,501,1,-0.982145,-0.021472,-0.218573,0.063841,0.110760,0.061010,-0.836035,0.180957,...,-1.147290,-0.435913,-0.679614,0.249326,-0.571290,-0.128164,1.005442,0.070684,150.816332,-0.010199
1,501,1,-1.003945,-0.055020,-0.150732,0.114290,0.075176,0.083377,-0.817676,0.066455,...,-1.346973,-0.321582,-0.670654,0.542167,-0.378275,-0.520994,1.038137,0.168264,155.720618,0.120175
2,501,1,-0.994657,-0.041391,-0.161117,0.067852,0.107584,0.137541,-0.888013,0.177832,...,-1.148071,-0.241113,-0.689697,0.012539,-0.447214,-0.424962,1.002341,0.073532,150.351206,-0.222950
3,501,1,-1.005066,-0.086511,-0.170156,0.105186,0.082683,0.085037,-0.859424,0.073731,...,-1.314575,-0.278320,-0.684375,0.127624,-0.448272,-0.365001,1.038057,0.152563,155.708485,-0.262919
4,501,1,-0.999507,-0.084847,-0.207869,0.091950,0.081913,0.078310,-0.856299,0.057519,...,-1.287256,-0.268579,-0.648730,0.185175,-0.464170,-0.494353,1.007433,0.124515,151.114953,0.433201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,517,4,-0.949793,0.032963,-0.334052,0.002416,0.001694,0.004774,-0.944580,0.034400,...,0.093457,-0.094165,-1.087427,-0.366528,0.095066,-0.231637,1.091659,0.001750,163.748886,-0.001836
10679,517,4,-0.949345,0.033850,-0.337324,0.002466,0.000909,0.002074,-0.943286,0.034644,...,0.093530,-0.087768,-1.090332,-0.226221,0.216941,0.339454,1.092114,0.002622,163.817049,0.001032
10680,517,4,-0.948054,0.034105,-0.338662,0.003087,0.001960,0.002550,-0.941992,0.039429,...,0.092139,-0.085791,-1.090845,0.441930,0.213509,0.461322,1.091379,0.002161,163.706816,0.000654
10681,517,5,-0.170341,0.124614,0.973431,0.041157,0.078031,0.022701,-0.090772,0.290698,...,0.032226,-0.293774,-1.168750,-0.559083,0.705100,-0.088620,1.092086,0.041754,163.812865,0.127425


In [10]:
og_data["label"].value_counts()

label
1    6773
3    2390
4     572
5     539
2     409
Name: count, dtype: int64

## Filter Method

* mafese only works with arrays, so we have to convert our df to this format 

In [11]:
og_data_values = og_data.values
og_data_values

array([[ 5.01000000e+02,  1.00000000e+00, -9.82145326e-01, ...,
         7.06840309e-02,  1.50816332e+02, -1.01994833e-02],
       [ 5.01000000e+02,  1.00000000e+00, -1.00394480e+00, ...,
         1.68263975e-01,  1.55720618e+02,  1.20174706e-01],
       [ 5.01000000e+02,  1.00000000e+00, -9.94656729e-01, ...,
         7.35323502e-02,  1.50351206e+02, -2.22949890e-01],
       ...,
       [ 5.17000000e+02,  4.00000000e+00, -9.48054119e-01, ...,
         2.16141091e-03,  1.63706816e+02,  6.53726274e-04],
       [ 5.17000000e+02,  5.00000000e+00, -1.70341209e-01, ...,
         4.17536473e-02,  1.63812865e+02,  1.27425261e-01],
       [ 5.17000000e+02,  5.00000000e+00, -1.99301906e-01, ...,
         9.38459773e-03,  1.64758329e+02,  8.49962934e-03]],
      shape=(10683, 40))

* we define our input and outputs

In [12]:
X = og_data_values[:, 2:]
y = og_data_values[:, 1]

In [13]:
data = Data(X,y)

* divide by train and test set

In [14]:
data.split_train_test(test_size = 0.2, inplace = True) # 80% train y 20% test
print(data.X_train[:2].shape)
print(data.y_train[:2].shape)

(2, 38)
(2,)


* normalize our input data, not the output

In [15]:
data.X_train, scaler_X = data.scale(data.X_train, scaling_methods=("standard", "minmax"))
data.X_test = scaler_X.transform(data.X_test)

* define our classifier and filter methods availables on mafese 

In [16]:
classifiers = {
    "knn": KNeighborsClassifier(),
    "svm": SVC()
}

filter_methods = ['CHI', 'ANOVA', 'MI', 'KENDALL', 'SPEARMAN']

In [ ]:
# results list 
filter_results = []

# number of characteristics to select, we defined this value by the avergae number of
# characterictics the wrapper method selected to a fair comparison
n_features = 6

# number of iterations
n_iterationes = 30

# Evaluation with each feature selection method
for method in filter_methods:
    print(f"\n*** Evaluating with selection method: {method} ***")

    for iteration in range (n_iterationes):

      print(f"\n--- Iteration {iteration + 1} ---")

      # Apply the feature selection method
      feat_selector = FilterSelector(problem="classification", method=method, n_features=n_features)
      feat_selector.fit(data.X_train, data.y_train)

      # Get the selected features
      selected_features = feat_selector.selected_feature_indexes
      print("Solution features: ", feat_selector.selected_feature_solution)
      print("Selected feature index:", feat_selector.selected_feature_indexes)
      print("Selected feature names:", og_data.columns[2:][feat_selector.selected_feature_indexes])

      # We filter the selected features in the training and test sets
      X_train_selected = feat_selector.transform(data.X_train)
      X_test_selected = feat_selector.transform(data.X_test)

      # we evaluate with each classifier
      for clf in classifiers:
          classifier = classifiers[clf]

          # we measure the time of training and evaluation
          start_time = time.time()
          classifier.fit(X_train_selected, data.y_train)
          y_pred = classifier.predict(X_test_selected)
          total_time = time.time() - start_time

          # calculate metrics 
          accuracy = accuracy_score(data.y_test, y_pred)
          precision = precision_score(data.y_test, y_pred, average='macro')
          recall = recall_score(data.y_test, y_pred, average='macro')
          F1 = f1_score(data.y_test, y_pred, average='macro')
          MCC = matthews_corrcoef(data.y_test, y_pred)
          Balanced_accuracy = balanced_accuracy_score(data.y_test, y_pred)

          # save results
          filter_results.append({
              "filter_method": method,
              "classifier": clf,
              "accuracy": accuracy,
              "precision": precision,
              "recall": recall,
              "F1": F1,
              "MCC": MCC,
              "Balanced_accuracy": Balanced_accuracy,
              "total_time": total_time,
              "features": list(og_data.columns[2:][selected_features])
              })

          print(f"Results for {clf} with {method}:")
          print(f"Accuracy Score: {accuracy}")
          print(f"Precision Score: {precision}")
          print(f"Recall Score: {recall}")
          print(f"F1 Score: {F1}")
          print(f"MCC Score: {MCC}")
          print(f"Balanced Accuracy Score: {Balanced_accuracy}")
          print(f"Total time: {total_time:.2f} seconds")
          print("-" * 30)

# Convert the results to a data frame and save them
df_filter_results = pd.DataFrame(filter_results)
df_filter_results.to_csv("filter_method_results.csv", index=False)


*** Evaluating with selection method: CHI ***

--- Iteration 1 ---
Solution features:  [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0]
Selected feature indices: [ 0  6 19 22 24 35]
Selected feature names: Index(['meanX', 'maxX', 'meanX2', 'sdX2', 'sdZ2', 'sdMagnitude2'], dtype='object')
Results for knn with CHI:
Accuracy Score: 0.9499298081422555
Precision Score: 0.8767890843071644
Recall Score: 0.8621004109399987
F1 Score: 0.8689810705290583
MCC Score: 0.9072945946808285
Balanced Accuracy Score: 0.8621004109399987
Total time: 0.54 seconds
------------------------------
Results for svm with CHI:
Accuracy Score: 0.9499298081422555
Precision Score: 0.8502050228681121
Recall Score: 0.8075849589770051
F1 Score: 0.8126943609488618
MCC Score: 0.9071882548346071
Balanced Accuracy Score: 0.8075849589770051
Total time: 0.96 seconds
------------------------------

--- Iteration 2 ---
Solution features:  [1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 

c:\Users\Coord-Personal\Documents\ITZELT\data_sciece_girlypop\proyecto_salvador\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Results for svm with ANOVA:
Accuracy Score: 0.9480580252690688
Precision Score: 0.7681824520946577
Recall Score: 0.786642813627177
F1 Score: 0.7771336053868743
MCC Score: 0.9040074792115378
Balanced Accuracy Score: 0.786642813627177
Total time: 0.81 seconds
------------------------------

--- Iteration 2 ---
Solution features:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0
 0]
Selected feature indices: [ 0 19 21 25 27 28]
Selected feature names: Index(['meanX', 'meanX2', 'meanZ2', 'maxX2', 'maxZ2', 'minX2'], dtype='object')
Results for knn with ANOVA:
Accuracy Score: 0.9513336452971455
Precision Score: 0.8839039323135142
Recall Score: 0.8439865484184533
F1 Score: 0.8582328732378643
MCC Score: 0.9095785565638915
Balanced Accuracy Score: 0.8439865484184533
Total time: 0.20 seconds
------------------------------


c:\Users\Coord-Personal\Documents\ITZELT\data_sciece_girlypop\proyecto_salvador\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Results for svm with ANOVA:
Accuracy Score: 0.9480580252690688
Precision Score: 0.7681824520946577
Recall Score: 0.786642813627177
F1 Score: 0.7771336053868743
MCC Score: 0.9040074792115378
Balanced Accuracy Score: 0.786642813627177
Total time: 0.92 seconds
------------------------------

--- Iteration 3 ---
Solution features:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0
 0]
Selected feature indices: [ 0 19 21 25 27 28]
Selected feature names: Index(['meanX', 'meanX2', 'meanZ2', 'maxX2', 'maxZ2', 'minX2'], dtype='object')
Results for knn with ANOVA:
Accuracy Score: 0.9513336452971455
Precision Score: 0.8839039323135142
Recall Score: 0.8439865484184533
F1 Score: 0.8582328732378643
MCC Score: 0.9095785565638915
Balanced Accuracy Score: 0.8439865484184533
Total time: 0.20 seconds
------------------------------


...

c:\Users\Coord-Personal\Documents\ITZELT\data_sciece_girlypop\proyecto_salvador\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Results for svm with SPEARMAN:
Accuracy Score: 0.9438465138043987
Precision Score: 0.7657542487182185
Recall Score: 0.7831385001693052
F1 Score: 0.774126035801836
MCC Score: 0.8960688329485713
Balanced Accuracy Score: 0.7831385001693052
Total time: 1.23 seconds
------------------------------

--- Iteration 29 ---
Solution features:  [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0]
Selected feature indices: [ 9 10 11 19 28 29]
Selected feature names: Index(['minX', 'minY', 'minZ', 'meanX2', 'minX2', 'minY2'], dtype='object')
Results for knn with SPEARMAN:
Accuracy Score: 0.941974730931212
Precision Score: 0.852789976619891
Recall Score: 0.832370352190253
F1 Score: 0.8408032663684544
MCC Score: 0.8923245300389314
Balanced Accuracy Score: 0.832370352190253
Total time: 0.37 seconds
------------------------------


c:\Users\Coord-Personal\Documents\ITZELT\data_sciece_girlypop\proyecto_salvador\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Results for svm with SPEARMAN:
Accuracy Score: 0.9438465138043987
Precision Score: 0.7657542487182185
Recall Score: 0.7831385001693052
F1 Score: 0.774126035801836
MCC Score: 0.8960688329485713
Balanced Accuracy Score: 0.7831385001693052
Total time: 1.17 seconds
------------------------------

--- Iteration 30 ---
Solution features:  [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0
 0]
Selected feature indices: [ 9 10 11 19 28 29]
Selected feature names: Index(['minX', 'minY', 'minZ', 'meanX2', 'minX2', 'minY2'], dtype='object')
Results for knn with SPEARMAN:
Accuracy Score: 0.941974730931212
Precision Score: 0.852789976619891
Recall Score: 0.832370352190253
F1 Score: 0.8408032663684544
MCC Score: 0.8923245300389314
Balanced Accuracy Score: 0.832370352190253
Total time: 0.27 seconds
------------------------------
Results for svm with SPEARMAN:
Accuracy Score: 0.9438465138043987
Precision Score: 0.7657542487182185
Recall Score: 0.7831385001693052
F1 Score: 0.774

c:\Users\Coord-Personal\Documents\ITZELT\data_sciece_girlypop\proyecto_salvador\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

